In [188]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
from functools import reduce
import operator
import json
from pymongo import MongoClient
from pymongo import GEOSPHERE
from cartoframes.viz import Map, Layer, popup_element
import geopandas as gpd
import sys
sys.path.append('../')
from bson.json_util import dumps
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import src.functions as func

### Choosing 3 cities to place my office:

#### A. Madrid:

In [2]:
madrid=func.geocode("Castellana 89, Paseo de la Castellana, Madrid") #Torre Picasso.

In [3]:
madrid

{'type': 'Point', 'coordinates': ['40.44980', '-3.69164']}

#### B. Málaga:

In [6]:
malaga=func.geocode("Calle Larios, Malaga")

In [7]:
malaga

{'type': 'Point', 'coordinates': ['36.72048', '-4.42166']}

#### C. Barcelona:

In [8]:
barcelona=func.geocode("Plaça de Catalunya, Barcelona")

In [9]:
barcelona

{'type': 'Point', 'coordinates': ['41.38781', '2.16920']}

### Prioritizing some requirements over othes:

- **Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.**
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- **Executives like Starbucks A LOT. Ensure there's a starbucks not too far.**
- **Account managers need to travel a lot.**
- **Everyone in the company is between 25 and 40, give them some place to go party.**
- **The CEO is vegan.**
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

### API Foursquare requests:

In [10]:
load_dotenv()

True

In [11]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [12]:
url = "https://favqs.com/api/session"
url_query = 'https://api.foursquare.com/v2/venues/search'

#### A. Madrid-Empresas de diseño:

In [13]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "empresas de diseño"
}

In [14]:
dis_mad = requests.get(url_query, params = parametros).json()

In [15]:
diseños_md = dis_mad["response"]["venues"]

In [16]:
lista_diseñomad=func.extraetodo(diseños_md)

In [17]:
datos_diseñomadrid = pd.DataFrame(lista_diseñomad)


In [18]:
#datos_diseñomadrid

In [19]:
diseño_madlimpio=datos_diseñomadrid.drop([0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,17,18,19,20,21,23,24,25,28,29],axis=0)

In [20]:
diseño_madlimpio["categoria"] = "Diseño"
diseño_madlimpio.head()

,nombre,latitud,longitud,location,categoria
7,Vivero de empresas de Moratalaz. Madrid Emprende,40.409162,-3.648840,"{'type': 'Point', 'coordinates': [40.409162199...",Diseño
16,Vivero de empresas de Puente de Vallecas. Madr...,40.383264,-3.656344,"{'type': 'Point', 'coordinates': [40.383264187...",Diseño
22,Preincubadora de empresas - Madrid Crece,40.447657,-3.714246,"{'type': 'Point', 'coordinates': [40.447657094...",Diseño
26,Arquimedes Llorens Atelier De Diseño De Moda Y...,40.424150,-3.706481,"{'type': 'Point', 'coordinates': [40.42415, -3...",Diseño
27,Instituto de Empresas de Madrid,40.423290,-3.707628,"{'type': 'Point', 'coordinates': [40.423290252...",Diseño


In [21]:
diseño_madlimpio.shape

(5, 5)

In [22]:
diseñoMadrid = diseño_madlimpio.copy()

#### A. Madrid-Starbucks:

In [23]:
parametros_St = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Starbucks"
}

In [24]:
St_mad = requests.get(url_query, params = parametros_St).json()

In [25]:
#St_mad

In [26]:
St_datamd = St_mad["response"]["venues"]

In [27]:
lista_starbucksmad=func.extraetodo(St_datamd)

In [28]:
datos_Starbucksmadrid=pd.DataFrame(lista_starbucksmad)

In [29]:
#datos_Starbucksmadrid

In [30]:
datos_Starbucksmadrid["categoria"] = "Starbucks"
datos_Starbucksmadrid.head()

,nombre,latitud,longitud,location,categoria
0,Starbucks,40.447489,-3.693093,"{'type': 'Point', 'coordinates': [40.447488723...",Starbucks
1,Starbucks,40.447729,-3.695175,"{'type': 'Point', 'coordinates': [40.447728697...",Starbucks
2,Starbucks,40.455046,-3.693885,"{'type': 'Point', 'coordinates': [40.455046, -...",Starbucks
3,Starbucks,40.432968,-3.683220,"{'type': 'Point', 'coordinates': [40.432967606...",Starbucks
4,Starbucks,40.433564,-3.686829,"{'type': 'Point', 'coordinates': [40.433563917...",Starbucks


#### A. Madrid-Airports:

In [31]:
parametros_A = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Airport"
}

In [32]:
airport_mad=requests.get(url_query, params=parametros_A).json()

In [33]:
#airport_mad

In [34]:
air_datamd = airport_mad["response"]["venues"]

In [35]:
lista_airportmad=func.extraetodo(air_datamd)

In [36]:
datos_airportmadrid=pd.DataFrame(lista_airportmad)

In [37]:
aeropuerto_madrid=datos_airportmadrid.drop([0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,24,25,26,27,28,29])

In [38]:
aeropuerto_madrid

,nombre,latitud,longitud,location
4,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
17,Club de Madrid,40.415357,-3.711218,"{'type': 'Point', 'coordinates': [40.415357277..."
23,Sala VIP Cibeles,40.462628,-3.570200,"{'type': 'Point', 'coordinates': [40.462628412..."


In [39]:
aeropuerto_madrid["categoria"] = "aeropuerto"
aeropuerto_madrid.head()

,nombre,latitud,longitud,location,categoria
4,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162...",aeropuerto
17,Club de Madrid,40.415357,-3.711218,"{'type': 'Point', 'coordinates': [40.415357277...",aeropuerto
23,Sala VIP Cibeles,40.462628,-3.570200,"{'type': 'Point', 'coordinates': [40.462628412...",aeropuerto


#### A. Madrid-Disco:

In [40]:
parametros_D = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Disco"
}

In [41]:
disco_mad=requests.get(url_query,params=parametros_D).json()

In [42]:
disco_datamad=disco_mad["response"]["venues"]

In [43]:
lista_discomad=func.extraetodo(disco_datamad)

In [44]:
datos_discomadrid=pd.DataFrame(lista_discomad)

In [45]:
#datos_discomadrid

In [46]:
datos_discomadrid["categoria"] = "Disco"
datos_discomadrid.head()

,nombre,latitud,longitud,location,categoria
0,Disco Rug & Play,40.451888,-3.696266,"{'type': 'Point', 'coordinates': [40.451888, -...",Disco
1,Panamericana Disco,40.452806,-3.690264,"{'type': 'Point', 'coordinates': [40.452806048...",Disco
2,Disco Express,40.434539,-3.710257,"{'type': 'Point', 'coordinates': [40.434538914...",Disco
3,Disco Yedra Pub,40.434041,-3.714618,"{'type': 'Point', 'coordinates': [40.434040912...",Disco
4,Destellos Pub Disco,40.437682,-3.709905,"{'type': 'Point', 'coordinates': [40.437681716...",Disco


####  A. Madrid-Vegan:

In [47]:
parametros_V= {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Vegan"
}

In [48]:
vegan_mad=requests.get(url_query,params=parametros_V).json()

In [49]:
vegan_datamad=vegan_mad["response"]["venues"]

In [50]:
lista_veganmad=func.extraetodo(vegan_datamad)

In [51]:
datos_veganmadrid=pd.DataFrame(lista_veganmad)

In [52]:
vegan_madridlimpio=datos_veganmadrid.drop([15,29])

In [53]:
#vegan_madridlimpio

In [54]:
vegan_madridlimpio["categoria"] = "Vegan"
vegan_madridlimpio.head()

,nombre,latitud,longitud,location,categoria
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,"{'type': 'Point', 'coordinates': [40.4424564, ...",Vegan
1,Vegan Streets Madrid Centro,40.417512,-3.697636,"{'type': 'Point', 'coordinates': [40.4175117, ...",Vegan
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -...",Vegan
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -...",Vegan
4,Los Andenes Vegan,40.435206,-3.710348,"{'type': 'Point', 'coordinates': [40.435206, -...",Vegan


#### Suming up all requirements in a single list (Madrid).

In [55]:
MADRID_df=func.concat_requirements(diseño_madlimpio, datos_Starbucksmadrid, aeropuerto_madrid, datos_discomadrid, vegan_madridlimpio)

In [56]:
MADRID_df["ciudad"] = "Madrid"

In [57]:
MADRID_df.shape

(96, 6)

In [58]:
MADRID_df.sample(10)

,nombre,latitud,longitud,location,categoria,ciudad
7,Starbucks,40.425760,-3.691782,"{'type': 'Point', 'coordinates': [40.425760357...",Starbucks,Madrid
0,Starbucks,40.447489,-3.693093,"{'type': 'Point', 'coordinates': [40.447488723...",Starbucks,Madrid
20,Vegania Veggie Bar,40.385241,-3.747971,"{'type': 'Point', 'coordinates': [40.385241, -...",Vegan,Madrid
26,Starbucks,40.497687,-3.660368,"{'type': 'Point', 'coordinates': [40.497687216...",Starbucks,Madrid
16,Vivero de empresas de Puente de Vallecas. Madr...,40.383264,-3.656344,"{'type': 'Point', 'coordinates': [40.383264187...",Diseño,Madrid
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -...",Vegan,Madrid
26,Arquimedes Llorens Atelier De Diseño De Moda Y...,40.424150,-3.706481,"{'type': 'Point', 'coordinates': [40.42415, -3...",Diseño,Madrid
17,Starbucks,40.423113,-3.700798,"{'type': 'Point', 'coordinates': [40.423112989...",Starbucks,Madrid
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -...",Vegan,Madrid
16,La Vegana de Madrid,40.457386,-3.651415,"{'type': 'Point', 'coordinates': [40.457386040...",Vegan,Madrid


In [59]:
lista_madrid = MADRID_df.to_dict(orient = "records")

#### B. Málaga-Empresas de diseño:

In [60]:
parametros_mlg = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "empresas de diseño"
}

In [61]:
dis_mlg=requests.get(url_query,params=parametros_mlg).json()

In [62]:
#mlg_dis

In [63]:
diseños_mlg = dis_mlg["response"]["venues"]

In [64]:
lista_diseñomlg=func.extraetodo(diseños_mlg)

In [65]:
datos_diseñomlg=pd.DataFrame(lista_diseñomlg)

In [66]:
diseño_mlglimpio=datos_diseñomlg.drop([3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19])

In [67]:
diseño_mlglimpio

,nombre,latitud,longitud,location
0,Empresas de reformas,36.721033,-4.410110,"{'type': 'Point', 'coordinates': [36.721033086..."
1,Tango Estudio de diseño,36.721188,-4.424591,"{'type': 'Point', 'coordinates': [36.721187882..."
2,Medio Melón Estudio de Diseño Gráfico,36.723841,-4.418049,"{'type': 'Point', 'coordinates': [36.723840858..."
8,Tuimagina. Estudio de publicidad & Diseño,36.716835,-4.455073,"{'type': 'Point', 'coordinates': [36.716835393..."


In [68]:
diseño_mlglimpio["categoria"] = "Diseño"
diseño_mlglimpio.head()

,nombre,latitud,longitud,location,categoria
0,Empresas de reformas,36.721033,-4.410110,"{'type': 'Point', 'coordinates': [36.721033086...",Diseño
1,Tango Estudio de diseño,36.721188,-4.424591,"{'type': 'Point', 'coordinates': [36.721187882...",Diseño
2,Medio Melón Estudio de Diseño Gráfico,36.723841,-4.418049,"{'type': 'Point', 'coordinates': [36.723840858...",Diseño
8,Tuimagina. Estudio de publicidad & Diseño,36.716835,-4.455073,"{'type': 'Point', 'coordinates': [36.716835393...",Diseño


#### B. Málaga-Aeropuerto:

In [69]:
parametros_mlg_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "airport"
}

In [70]:
air_mlg=requests.get(url_query,params=parametros_mlg_air).json()

In [71]:
airport_mlg = air_mlg["response"]["venues"]

In [72]:
lista_airportmlg=func.extraetodo(airport_mlg)

In [73]:
datos_airmlg=pd.DataFrame(lista_airportmlg)

In [74]:
aeropuerto_mlglimpio=datos_airmlg.iloc[2:3]

In [75]:
aeropuerto_mlglimpio

,nombre,latitud,longitud,location
2,Aeropuerto de Málaga - Costa del Sol (AGP) (Ae...,36.677056,-4.492091,"{'type': 'Point', 'coordinates': [36.677056143..."


In [76]:
aeropuerto_mlglimpio["categoria"] = "Aeropuerto"
aeropuerto_mlglimpio.head()

<ipython-input-76-73d78663ccab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aeropuerto_mlglimpio["categoria"] = "Aeropuerto"


,nombre,latitud,longitud,location,categoria
2,Aeropuerto de Málaga - Costa del Sol (AGP) (Ae...,36.677056,-4.492091,"{'type': 'Point', 'coordinates': [36.677056143...",Aeropuerto


#### B. Málaga-Starbucks:

In [77]:
parametros_mlg_st = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "Starbucks"
}

In [78]:
st_mlg=requests.get(url_query,params=parametros_mlg_st).json()

In [79]:
Starbucks_mlg = st_mlg["response"]["venues"]

In [80]:
lista_Starbucks_mlg=func.extraetodo(Starbucks_mlg)

In [81]:
starbucks_dfmlg=pd.DataFrame(lista_Starbucks_mlg)

In [82]:
starbucks_dfmlg["categoria"] = "Starbucks"
starbucks_dfmlg.head()

,nombre,latitud,longitud,location,categoria
0,Starbucks Félix Saenz,36.719121,-4.423158,"{'type': 'Point', 'coordinates': [36.719120753...",Starbucks
1,Starbucks ECI Málaga Hilera,36.718070,-4.429669,"{'type': 'Point', 'coordinates': [36.718070373...",Starbucks
2,Starbucks McArthur Glen,36.658077,-4.477995,"{'type': 'Point', 'coordinates': [36.658077, -...",Starbucks
3,Starbucks,36.677194,-4.493945,"{'type': 'Point', 'coordinates': [36.677194301...",Starbucks
4,Starbucks,36.676346,-4.491327,"{'type': 'Point', 'coordinates': [36.676345929...",Starbucks


#### B. Málaga-Disco:

In [83]:
parametros_mlg_disco = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "disco"
}

In [84]:
disco_mlg=requests.get(url_query,params=parametros_mlg_disco).json()

In [85]:
disco_mlg = disco_mlg["response"]["venues"]

In [86]:
lista_disco_mlg=func.extraetodo(disco_mlg)

In [87]:
disco_dfmlg=pd.DataFrame(lista_disco_mlg)

In [88]:
disco_mlglimpio=disco_dfmlg.drop([4,6,9,12])

In [89]:
disco_mlglimpio["categoria"] = "Disco"
disco_mlglimpio.head()

,nombre,latitud,longitud,location,categoria
0,Disco Bronco,36.714944,-4.424100,"{'type': 'Point', 'coordinates': [36.714944211...",Disco
1,Clasic Disco Club,36.719222,-4.411931,"{'type': 'Point', 'coordinates': [36.719222, -...",Disco
2,Discos Candilejas,36.721888,-4.421745,"{'type': 'Point', 'coordinates': [36.721887588...",Disco
3,Disco-Sala La Farándula,36.727779,-4.394269,"{'type': 'Point', 'coordinates': [36.727779382...",Disco
5,City Locker / Discoveringmalaga,36.719645,-4.423855,"{'type': 'Point', 'coordinates': [36.719645, -...",Disco


#### B. Málaga-Vegan:

In [90]:
parametros_mlg_vegan = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{malaga['coordinates'][0]}, {malaga['coordinates'][1]}",
    "query": "Vegan"
}

In [91]:
vegan_mlg=requests.get(url_query,params=parametros_mlg_vegan).json()

In [92]:
vegan_mlg = vegan_mlg["response"]["venues"]

In [93]:
lista_vegan_mlg=func.extraetodo(vegan_mlg)

In [94]:
vegan_mlgdf=pd.DataFrame(lista_vegan_mlg)

In [95]:
vegan_mlgdf["categoria"] = "Disco"
vegan_mlgdf.head()

,nombre,latitud,longitud,location,categoria
0,Mimo Vegan Bistro,36.716062,-4.424227,"{'type': 'Point', 'coordinates': [36.716062, -...",Disco
1,Vegan La Siria,36.714369,-4.428553,"{'type': 'Point', 'coordinates': [36.714369, -...",Disco
2,Veganized - non toxic fashion,36.720366,-4.424072,"{'type': 'Point', 'coordinates': [36.720366, -...",Disco
3,Vegan Place,36.691046,-4.447781,"{'type': 'Point', 'coordinates': [36.691046, -...",Disco
4,La Vegana,36.724179,-4.418461,"{'type': 'Point', 'coordinates': [36.724179, -...",Disco


#### Suming up all requirements in a single list (Málaga).

In [253]:
MALAGA_df=func.concat_requirements(diseño_mlglimpio, aeropuerto_mlglimpio, starbucks_dfmlg, disco_mlglimpio, vegan_mlgdf)

In [254]:
MALAGA_df["ciudad"] = "Málaga"

In [255]:
lista_malaga = MALAGA_df.to_dict(orient = "records")

#### C. Barcelona-Design:

In [99]:
parametros_bcn_dis = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Diseño"
}

In [100]:
dis_bcn=requests.get(url_query,params=parametros_bcn_dis).json()

diseño_bcn = dis_bcn["response"]["venues"]

lista_diseñobcn=func.extraetodo(diseño_bcn)
diseño_bcndf=pd.DataFrame(lista_diseñobcn)

In [101]:
diseño_bcndf=diseño_bcndf.drop([1,2,5,11,18,19,21,22])

In [102]:
diseño_bcndf["categoria"] = "Diseño"
diseño_bcndf.head()

,nombre,latitud,longitud,location,categoria
0,Tuctucbarcelona —Diseño Gráfico y Web—,41.386273,2.174185,"{'type': 'Point', 'coordinates': [41.386273475...",Diseño
3,Diseño Paginas Web Yá Barcelona,41.380311,2.169969,"{'type': 'Point', 'coordinates': [41.380311, 2...",Diseño
4,"RC2 Ingenieria y Diseño, S.L",41.390638,2.174306,"{'type': 'Point', 'coordinates': [41.390638180...",Diseño
6,Dulce Diseño El Born,41.388119,2.178501,"{'type': 'Point', 'coordinates': [41.388118801...",Diseño
7,Socialweb Diseño Web Barcelona,41.377837,2.129298,"{'type': 'Point', 'coordinates': [41.377837033...",Diseño


#### C. Barcelona-Airport:

In [103]:
parametros_bcn_air = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Airport"
}

In [104]:
air_bcn=requests.get(url_query,params=parametros_bcn_air).json()

airport_bcn = air_bcn["response"]["venues"]

lista_airportbcn=func.extraetodo(airport_bcn)

In [105]:
air_bcndf=pd.DataFrame(lista_airportbcn)

In [106]:
air_bcnlimpio=air_bcndf.head(1)

In [107]:
air_bcnlimpio["categoria"] = "Aeropuerto"
air_bcnlimpio.head()

<ipython-input-107-1823c9e4b61a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_bcnlimpio["categoria"] = "Aeropuerto"


,nombre,latitud,longitud,location,categoria
0,Barcelona–El Prat Josep Tarradellas Airport (B...,41.295607,2.076223,"{'type': 'Point', 'coordinates': [41.295606993...",Aeropuerto


#### C. Barcelona-Starbucks:

In [108]:
parametros_bcn_st = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Starbucks"
}

In [109]:
st_bcn=requests.get(url_query,params=parametros_bcn_st).json()

starbucks_bcn = st_bcn["response"]["venues"]

lista_starbucksbcn=func.extraetodo(starbucks_bcn)

In [110]:
starbucks_dfbcn=pd.DataFrame(lista_starbucksbcn)

In [111]:
starbucks_dfbcn["categoria"] = "Starbucks"
starbucks_dfbcn.head()

,nombre,latitud,longitud,location,categoria
0,Starbucks Rambla Cataluña,41.387954,2.167901,"{'type': 'Point', 'coordinates': [41.387953542...",Starbucks
1,Starbucks Pelai,41.385805,2.167858,"{'type': 'Point', 'coordinates': [41.385804589...",Starbucks
2,Starbucks Rambla Canaletas,41.383485,2.171382,"{'type': 'Point', 'coordinates': [41.383485318...",Starbucks
3,Starbucks Plaza Universitat,41.386418,2.165150,"{'type': 'Point', 'coordinates': [41.386417656...",Starbucks
4,Starbucks Sant Pere,41.388783,2.172063,"{'type': 'Point', 'coordinates': [41.388782617...",Starbucks


#### C. Barcelona-Disco:

In [112]:
parametros_bcn_disco = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Disco"
}

In [113]:
disco_bcn=requests.get(url_query,params=parametros_bcn_disco).json()

discoteca_bcn = disco_bcn["response"]["venues"]

lista_discotecabcn=func.extraetodo(discoteca_bcn)

In [114]:
disco_bcndf=pd.DataFrame(lista_discotecabcn)

In [115]:
disco_bcnlimpio=disco_bcndf.drop([8,21,22])

In [116]:
disco_bcnlimpio["categoria"] = "Disco"
disco_bcnlimpio.head()

,nombre,latitud,longitud,location,categoria
0,Dirty House Disco,41.387745,2.168554,"{'type': 'Point', 'coordinates': [41.387745, 2...",Disco
1,Discos Paradiso,41.382223,2.165922,"{'type': 'Point', 'coordinates': [41.382223232...",Disco
2,Disco Bus,41.389916,2.171634,"{'type': 'Point', 'coordinates': [41.389916, 2...",Disco
3,Disco Pub Joe's,41.377008,2.137311,"{'type': 'Point', 'coordinates': [41.377008007...",Disco
4,Lux Disco,41.391823,2.166363,"{'type': 'Point', 'coordinates': [41.391822926...",Disco


#### C. Barcelona-Vegan:

In [117]:
parametros_bcn_veg = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{barcelona['coordinates'][0]}, {barcelona['coordinates'][1]}",
    "query": "Vegan"
}

In [118]:
veg_bcn=requests.get(url_query,params=parametros_bcn_veg).json()

vegan_bcn = veg_bcn["response"]["venues"]

lista_veganbcn=func.extraetodo(vegan_bcn)

In [119]:
vegan_bcndf=pd.DataFrame(lista_veganbcn)

In [120]:
vegan_bcnlimpio=vegan_bcndf.drop([2,4,15])

In [121]:
vegan_bcnlimpio["categoria"] = "Vegan"
vegan_bcnlimpio.head()

,nombre,latitud,longitud,location,categoria
0,Vegan Junk Food Bar,41.383386,2.180255,"{'type': 'Point', 'coordinates': [41.383385646...",Vegan
1,All Vegan,41.382820,2.169548,"{'type': 'Point', 'coordinates': [41.38282, 2....",Vegan
3,ENJOY VEGAN,41.388152,2.180673,"{'type': 'Point', 'coordinates': [41.3881518, ...",Vegan
5,Monchito Vegan,41.380971,2.179880,"{'type': 'Point', 'coordinates': [41.380970639...",Vegan
6,Monchito Vegan,41.381030,2.179852,"{'type': 'Point', 'coordinates': [41.38103, 2....",Vegan


#### Suming up all requirements in a single list (Barcelona).

In [122]:
BARCELONA_df=func.concat_requirements(diseño_bcndf, air_bcnlimpio, starbucks_dfbcn, disco_bcnlimpio, vegan_bcnlimpio)


In [123]:
BARCELONA_df["ciudad"] = "Barcelona"

In [124]:
lista_bcn = BARCELONA_df.to_dict(orient = "records")

### Exporting data to Mongo DB Compass.

In [125]:
client = MongoClient("localhost:27017")
db = client.get_database("PROYECTO_3")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO_3')

#### A. Madrid:

In [126]:
coleccion1 = db["MADRID"]
coleccion1.insert_many(lista_madrid)

#### B. Málaga:

In [127]:
coleccion2 = db["MALAGA"]
coleccion2.insert_many(lista_malaga)

#### C. Barcelona:

In [128]:
coleccion3 = db["BARCELONA"]
coleccion3.insert_many(lista_bcn)

### Geoqueries:

#### A. Madrid:

In [129]:
MADRID = db.get_collection("MADRID") # Me conecto a la colección "MADRID".

In [130]:
MADRID

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO_3'), 'MADRID')

In [131]:
db.MADRID.create_index([("location", GEOSPHERE)]) # Creo el index 2dsphere.

'location_2dsphere'

In [132]:
query=[{
"$geoNear": {'near': [40.4498,-3.69164 ],
             'distanceField': 'distance',
             'maxDistance': 2000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

In [133]:
geoloc=MADRID.aggregate(query)

In [134]:
geoquery_madrid=json.loads(dumps(geoloc))

In [135]:
df_madrid = pd.DataFrame(geoquery_madrid)
type(df_madrid)
df_madrid.drop_duplicates(["latitud","longitud"],inplace=True)

In [136]:
df_madrid.head()

,_id,nombre,latitud,longitud,location,categoria,ciudad,distance
0,{'$oid': '6192843bf7bf3a2a12a2b49c'},Starbucks,40.447489,-3.693093,"{'type': 'Point', 'coordinates': [40.447488723...",Starbucks,Madrid,0.303146
8,{'$oid': '6192ad49103aea2c0e60108a'},Panamericana Disco,40.452806,-3.690264,"{'type': 'Point', 'coordinates': [40.452806048...",Disco,Madrid,0.366997
16,{'$oid': '6192898eef392ea590fd7a95'},la cubana discoteca,40.450147,-3.695196,"{'type': 'Point', 'coordinates': [40.450147, -...",Disco,Madrid,0.397279
24,{'$oid': '6192ac142ff6bc6486a18632'},Starbucks,40.447729,-3.695175,"{'type': 'Point', 'coordinates': [40.447728697...",Starbucks,Madrid,0.455384
32,{'$oid': '6192843bf7bf3a2a12a2b4bd'},Disco Rug & Play,40.451888,-3.696266,"{'type': 'Point', 'coordinates': [40.451888, -...",Disco,Madrid,0.564160


In [137]:
madrid_agrupado=df_madrid.groupby(["categoria"])["distance"].mean()

In [138]:
dfMADRID=pd.DataFrame(madrid_agrupado).reset_index()

In [139]:
dfMADRID

,categoria,distance
0,Disco,3.418366
1,Diseño,3.729330
2,Starbucks,3.912457
3,Vegan,4.830811
4,aeropuerto,11.155070


In [140]:
func.normalize(dfMADRID,"distance")

[0.0, 0.040193279286611386, 0.06386316385582932, 0.18256417926221383, 1.0]

In [141]:
dfMADRID["normalizado"]=func.normalize(dfMADRID,"distance")

In [142]:
dfMADRID

,categoria,distance,normalizado
0,Disco,3.418366,0.000000
1,Diseño,3.729330,0.040193
2,Starbucks,3.912457,0.063863
3,Vegan,4.830811,0.182564
4,aeropuerto,11.155070,1.000000


In [143]:
dfMADRID["weights"] = func.weights(dfMADRID)

In [144]:
dfMADRID

,categoria,distance,normalizado,weights
0,Disco,3.418366,0.000000,0.000000
1,Diseño,3.729330,0.040193,0.008039
2,Starbucks,3.912457,0.063863,0.025545
3,Vegan,4.830811,0.182564,0.018256
4,aeropuerto,11.155070,1.000000,0.100000


In [168]:
dfMADRID["ciudad"] = "Madrid"

In [169]:
dfMADRID

,categoria,distance,normalizado,weights,ciudad
0,Disco,3.418366,0.000000,0.000000,Madrid
1,Diseño,3.729330,0.040193,0.008039,Madrid
2,Starbucks,3.912457,0.063863,0.025545,Madrid
3,Vegan,4.830811,0.182564,0.018256,Madrid
4,aeropuerto,11.155070,1.000000,0.100000,Madrid


#### B. Málaga:

In [146]:
MALAGA = db.get_collection("MALAGA") # Me conecto a la colección "MALAGA".

MALAGA

In [147]:
db.MALAGA.create_index([("location", GEOSPHERE)]) # Creo el index 2dsphere.

'location_2dsphere'

In [148]:
query_mlg=[{
"$geoNear": {'near': [36.72048, -4.42166],
             'distanceField': 'distance',
             'maxDistance': 2000,
             'distanceMultiplier': 6371,
             'spherical':True}}]

In [149]:
geoloc_MLG=MALAGA.aggregate(query_mlg)

geoquery_malaga=json.loads(dumps(geoloc_MLG))

In [150]:
df_malaga = pd.DataFrame(geoquery_malaga)

In [151]:
df_malaga.drop_duplicates(["latitud","longitud"],inplace=True)
df_malaga


,_id,nombre,latitud,longitud,location,categoria,ciudad,distance
0,{'$oid': '61928f0fef392ea590fd7ada'},Discos Candilejas,36.721888,-4.421745,"{'type': 'Point', 'coordinates': [36.721887588...",Disco,Málaga,0.156336
5,{'$oid': '6192bcf97c6b9ec9dabfa26e'},Starbucks Félix Saenz,36.719121,-4.423158,"{'type': 'Point', 'coordinates': [36.719120753...",Starbucks,Málaga,0.224594
10,{'$oid': '6192aef3103aea2c0e6010de'},City Locker / Discoveringmalaga,36.719645,-4.423855,"{'type': 'Point', 'coordinates': [36.719645, -...",Disco,Málaga,0.261038
13,{'$oid': '6192aef3103aea2c0e6010f6'},Veganized - non toxic fashion,36.720366,-4.424072,"{'type': 'Point', 'coordinates': [36.720366, -...",Disco,Málaga,0.268500
18,{'$oid': '61928f0fef392ea590fd7adc'},Discoteca Andén,36.722607,-4.420381,"{'type': 'Point', 'coordinates': [36.722606922...",Disco,Málaga,0.275349
20,{'$oid': '6192b9357e820ce8b39b14fe'},Tango Estudio de diseño,36.721188,-4.424591,"{'type': 'Point', 'coordinates': [36.721187882...",Diseño,Málaga,0.335203
25,{'$oid': '61929272ef392ea590fd7b35'},La Vegana,36.724179,-4.418461,"{'type': 'Point', 'coordinates': [36.724179, -...",Disco,Málaga,0.542865
30,{'$oid': '6192aef3103aea2c0e6010c6'},Medio Melón Estudio de Diseño Gráfico,36.723841,-4.418049,"{'type': 'Point', 'coordinates': [36.723840858...",Diseño,Málaga,0.547782
35,{'$oid': '6192aef3103aea2c0e6010f4'},Mimo Vegan Bistro,36.716062,-4.424227,"{'type': 'Point', 'coordinates': [36.716062, -...",Disco,Málaga,0.566899
40,{'$oid': '61929272ef392ea590fd7b16'},Disco Bronco,36.714944,-4.424100,"{'type': 'Point', 'coordinates': [36.714944211...",Disco,Málaga,0.671013


In [152]:
malaga_agrupado=df_malaga.groupby(["categoria"])["distance"].mean()

dfMALAGA=pd.DataFrame(malaga_agrupado).reset_index()

dfMALAGA

,categoria,distance
0,Aeropuerto,9.192777
1,Disco,17.178973
2,Diseño,1.476519
3,Starbucks,21.139791


In [153]:
func.normalize(dfMALAGA,"distance")

dfMALAGA["normalizado"]=func.normalize(dfMALAGA,"distance")

dfMALAGA["weights"] = func.weights(dfMALAGA)

dfMALAGA

#luego group by por ciudad

,categoria,distance,normalizado,weights
0,Aeropuerto,9.192777,0.392420,0.039242
1,Disco,17.178973,0.798568,0.159714
2,Diseño,1.476519,0.000000,0.000000
3,Starbucks,21.139791,1.000000,0.400000


In [166]:
dfMALAGA["ciudad"] = "Malaga"

In [167]:
dfMALAGA

,categoria,distance,normalizado,weights,ciudad
0,Aeropuerto,9.192777,0.392420,0.039242,Malaga
1,Disco,17.178973,0.798568,0.159714,Malaga
2,Diseño,1.476519,0.000000,0.000000,Malaga
3,Starbucks,21.139791,1.000000,0.400000,Malaga


#### C. Barcelona:

In [154]:
BARCELONA = db.get_collection("BARCELONA") # Me conecto a la colección "BARCELONA".

BARCELONA

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO_3'), 'BARCELONA')

In [155]:
db.BARCELONA.create_index([("location", GEOSPHERE)]) # Creo el index 2dsphere.

'location_2dsphere'

In [156]:
query_bcn=[{
"$geoNear": {'near': [41.38781, 2.16920],
             'distanceField': 'distance',
             'maxDistance': 2000,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

In [157]:
geoloc_bcn=BARCELONA.aggregate(query_bcn)

geoquery_barcelona=json.loads(dumps(geoloc_bcn))

In [158]:
df_barcelona = pd.DataFrame(geoquery_barcelona)

df_barcelona.drop_duplicates(["latitud","longitud"],inplace=True)

In [159]:
df_barcelona.head()

barcelona_agrupado=df_barcelona.groupby(["categoria"])["distance"].mean()

In [160]:
dfBARCELONA=pd.DataFrame(barcelona_agrupado).reset_index()

dfBARCELONA

,categoria,distance
0,Aeropuerto,14.555272
1,Disco,2.021723
2,Diseño,2.334211
3,Starbucks,2.317484
4,Vegan,2.621971


In [161]:
func.normalize(dfBARCELONA,"distance")

dfBARCELONA["normalizado"]=func.normalize(dfBARCELONA,"distance")

dfBARCELONA


,categoria,distance,normalizado
0,Aeropuerto,14.555272,1.000000
1,Disco,2.021723,0.000000
2,Diseño,2.334211,0.024932
3,Starbucks,2.317484,0.023598
4,Vegan,2.621971,0.047891


In [162]:
dfBARCELONA["weights"] = func.weights(dfBARCELONA)

dfBARCELONA

#luego group by por ciudad

,categoria,distance,normalizado,weights
0,Aeropuerto,14.555272,1.000000,0.100000
1,Disco,2.021723,0.000000,0.000000
2,Diseño,2.334211,0.024932,0.004986
3,Starbucks,2.317484,0.023598,0.009439
4,Vegan,2.621971,0.047891,0.004789


In [164]:
dfBARCELONA["ciudad"] = "Barcelona"

In [165]:
dfBARCELONA

,categoria,distance,normalizado,weights,ciudad
0,Aeropuerto,14.555272,1.000000,0.100000,Barcelona
1,Disco,2.021723,0.000000,0.000000,Barcelona
2,Diseño,2.334211,0.024932,0.004986,Barcelona
3,Starbucks,2.317484,0.023598,0.009439,Barcelona
4,Vegan,2.621971,0.047891,0.004789,Barcelona


### Concateno los tres dataframes agrupados por categorías de cada ciudad:

In [171]:
concatenado=func.concat_ciudades(dfMADRID,dfMALAGA,dfBARCELONA)

In [172]:
concatenado

,categoria,distance,normalizado,weights,ciudad
0,Disco,3.418366,0.000000,0.000000,Madrid
1,Diseño,3.729330,0.040193,0.008039,Madrid
2,Starbucks,3.912457,0.063863,0.025545,Madrid
3,Vegan,4.830811,0.182564,0.018256,Madrid
4,aeropuerto,11.155070,1.000000,0.100000,Madrid
0,Aeropuerto,9.192777,0.392420,0.039242,Malaga
1,Disco,17.178973,0.798568,0.159714,Malaga
2,Diseño,1.476519,0.000000,0.000000,Malaga
3,Starbucks,21.139791,1.000000,0.400000,Malaga
0,Aeropuerto,14.555272,1.000000,0.100000,Barcelona


In [181]:
final=concatenado.groupby(["ciudad"])["weights"].sum()

In [186]:
ranking=pd.DataFrame(final).reset_index()

In [187]:
ranking

,ciudad,weights
0,Barcelona,0.119215
1,Madrid,0.151840
2,Malaga,0.598956


### Mapa MADRID:

In [235]:
madrid

{'type': 'Point', 'coordinates': ['40.44980', '-3.69164']}

In [236]:
lat=madrid["coordinates"][0]

In [240]:
long=madrid["coordinates"][1]

In [241]:
inicial_lat = float(lat)
inicial_long =float(long)

In [242]:
map_1 = Map(location = [inicial_lat,inicial_long], zoom_start = 15)
map_1

In [243]:
MADRID_df.categoria.unique()

array(['Diseño', 'Starbucks', 'aeropuerto', 'Disco', 'Vegan'],
      dtype=object)

In [244]:
for i,row in MADRID_df.iterrows():
    requirements = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Diseño":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black")
        
    elif row["categoria"] == "Starbucks":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
        
    elif row["categoria"] == "aeropuerto":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="camera",
                     icon_color="black")
    
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="heart",
                     icon_color="black")
    
    mark = Marker(**requirements, icon=icono)
    mark.add_to(map_1)

In [245]:
map_1

### Mapa Málaga:

In [246]:
malaga

{'type': 'Point', 'coordinates': ['36.72048', '-4.42166']}

In [247]:
lat=malaga["coordinates"][0]

In [248]:
long=malaga["coordinates"][1]

In [249]:
inicial_lat2 = float(lat)
inicial_long2 =float(long)

In [250]:
map_2 = Map(location = [inicial_lat2,inicial_long2], zoom_start = 15)
map_2

In [271]:
MALAGA_df.sample(10)

,nombre,latitud,longitud,location,categoria,ciudad
21,Disco Pub Yuka,36.766087,-4.709443,"{'type': 'Point', 'coordinates': [36.766086570...",Disco,Málaga
11,Meson Mudejar Vegan,36.838543,-3.990528,"{'type': 'Point', 'coordinates': [36.838543, -...",Disco,Málaga
7,Disco Pub Límite,36.928900,-4.530873,"{'type': 'Point', 'coordinates': [36.928899564...",Disco,Málaga
3,Disco-Sala La Farándula,36.727779,-4.394269,"{'type': 'Point', 'coordinates': [36.727779382...",Disco,Málaga
1,Clasic Disco Club,36.719222,-4.411931,"{'type': 'Point', 'coordinates': [36.719222, -...",Disco,Málaga
4,Starbucks,36.676346,-4.491327,"{'type': 'Point', 'coordinates': [36.676345929...",Starbucks,Málaga
5,City Locker / Discoveringmalaga,36.719645,-4.423855,"{'type': 'Point', 'coordinates': [36.719645, -...",Disco,Málaga
25,Studio Disco,36.590938,-4.604666,"{'type': 'Point', 'coordinates': [36.590938, -...",Disco,Málaga
2,Aeropuerto de Málaga - Costa del Sol (AGP) (Ae...,36.677056,-4.492091,"{'type': 'Point', 'coordinates': [36.677056143...",Aeropuerto,Málaga
28,Boss Disco,36.542150,-4.619486,"{'type': 'Point', 'coordinates': [36.542149620...",Disco,Málaga


In [269]:
for i,row in MALAGA_df.iterrows():
    requirements = {"location": [row["latitud"], row["longitud"]], "tooltip": row["categoria"]}
    
    if row["categoria"] == "Diseño":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black")
        
    elif row["categoria"] == "Starbucks":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")
        
    elif row["categoria"] == "Aeropuerto":
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="plane",
                     icon_color="black")
        
    elif row["categoria"] == "Disco":
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="camera",
                     icon_color="black")
    
    elif row["categoria"] == "Vegan":
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="heart",
                     icon_color="black")
    
    mark = Marker(**requirements, icon=icono)
    mark.add_to(map_2)

In [270]:
map_2